In [3]:
import torch
import math

In [4]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim


In [5]:
vocab_size = 100
n_embed = 64
max_tokens = 10


In [6]:
class PositionalEncoding(nn.Module):

    def __init__(self, d_model: int, dropout: float = 0.1, max_len: int = 5000):
        super().__init__()
        self.dropout = nn.Dropout(p=dropout)

        position = torch.arange(max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2)
                             * (-math.log(10000.0) / d_model))
        pe = torch.zeros(max_len, 1, d_model)
        pe[:, 0, 0::2] = torch.sin(position * div_term)
        pe[:, 0, 1::2] = torch.cos(position * div_term)
        self.register_buffer('pe', pe)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
        Arguments:
            x: Tensor, shape ``[seq_len, batch_size, embedding_dim]``
        """
        x = x + self.pe[:x.size(0)]
        return self.dropout(x)

In [7]:
class SimpleTransformersModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, dropout=0.1, num_heads = 8):
        super(SimpleTransformersModel, self).__init__()
        
        self.hidden_size = hidden_size
        self.input_size = input_size
        self.output_size = output_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.pos_encoder = PositionalEncoding(hidden_size)
        self.attention = nn.MultiheadAttention(hidden_size, num_heads, dropout=dropout)
        self.linear = nn.Linear(hidden_size, output_size)
        self.softmax = nn.Softmax(dim=1)
        self.ffwd = nn.Sequential([
            nn.Linear(hidden_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, hidden_size)
        ])
        self.add_norm = nn.Sequential([
            nn.LayerNorm(hidden_size),
            nn.ReLU()
        ])

    def forward(self, x):
        x = self.embedding(x)
        x = self.pos_encoder(x)
        x, _ = self.attention(x, x, x)
        x = self.add_norm(x)
        x = self.ffwd(x)
        x = self.linear(x)
        x = self.softmax(x)
        return x